<a href="https://colab.research.google.com/github/ganeshred/NLPAuthorshipAttributionNLM/blob/main/Task3_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.


In [ ]:
from torchtext.legacy import data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

# from torchtext.legacy import data
# from torchtext.legacy import datasets



# from torchtext.legacy import data
from torchtext.legacy import datasets

from torchtext.legacy.data import Field 
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator
from torchtext.vocab import FastText
from torchtext.vocab import CharNGram

import pandas as pd
import numpy as np

In [ ]:
embedding = FastText('simple')

.vector_cache/wiki.simple.vec: 293MB [00:01, 160MB/s]                           
100%|██████████| 111051/111051 [00:19<00:00, 5562.04it/s] 


In [ ]:
# embedding_charngram = CharNGram()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Task3.csv')
# df = df [['tex','']]
df.head()

,Generation,label,label_int
0,Flint man facing 10 dog fighting and drug deal...,grover,5
1,the 30 most troubling lines from donald trump ...,ctrl,0
2,Wakayama rafting is a type of traditional raft...,instructgpt,7
3,The movies that predicted an outbreak like cor...,grover,5
4,companies that pay taxes are allowed to use ta...,pplm,8


In [ ]:
# df = df.rename(columns={'class':'label'})
df = df[['Generation','label']]

In [ ]:
df[['Generation']] = df[['Generation']]. astype(str)

In [ ]:
df.head()

,Generation,label
0,Flint man facing 10 dog fighting and drug deal...,grover
1,the 30 most troubling lines from donald trump ...,ctrl
2,Wakayama rafting is a type of traditional raft...,instructgpt
3,The movies that predicted an outbreak like cor...,grover
4,companies that pay taxes are allowed to use ta...,pplm


In [ ]:
text_field = Field(
    sequential=True,
    tokenize='basic_english', 
    fix_length=500,
    lower=True
)

label_field = Field(sequential=False, use_vocab=False)

# sadly have to apply preprocess manually
preprocessed_text = df['Generation'].apply(
    lambda x: text_field.preprocess(x)
)

# load fastext simple embedding with 300d
text_field.build_vocab(
    preprocessed_text, 
    vectors='fasttext.simple.300d'
)

# get the vocab instance
vocab = text_field.vocab

In [ ]:
# known token, in my case print 12
print(vocab['are'])
# unknown token, will print 0
print(vocab['crazy'])

21
1242


In [ ]:
d = enumerate(df['label'].unique())

In [ ]:
for i,l in d:
  print(i,l)

0 grover
1 ctrl
2 instructgpt
3 pplm
4 gpt2
5 xlnet
6 gpt
7 xlm
8 human
9 gpt3
10 fair


In [ ]:
ltoi = {l: i for i, l in enumerate(df['label'].unique())}


In [ ]:
ltoi

{'grover': 0,
 'ctrl': 1,
 'instructgpt': 2,
 'pplm': 3,
 'gpt2': 4,
 'xlnet': 5,
 'gpt': 6,
 'xlm': 7,
 'human': 8,
 'gpt3': 9,
 'fair': 10}

In [ ]:
# we still have to manually handle conversion from categorical to int
# ltoi = {l: i for i, l in }
df['label1'] = df['label'].apply(lambda y: ltoi[y])

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [ ]:
df.head(20)
# df1=df[['Generation','label1']]

,Generation,label,label1
0,Flint man facing 10 dog fighting and drug deal...,grover,0
1,the 30 most troubling lines from donald trump ...,ctrl,1
2,Wakayama rafting is a type of traditional raft...,instructgpt,2
3,The movies that predicted an outbreak like cor...,grover,0
4,companies that pay taxes are allowed to use ta...,pplm,3
5,1 A free press is essential to a functioning d...,instructgpt,2
6,s Saturday rally1 Research the venue Examine a...,instructgpt,2
7,The COVID 19 pandemic is an ongoing pandemic o...,instructgpt,2
8,my search for the boy in a child abuse video m...,gpt2,4
9,trump obeys seinfeld s law of use 5 7 16 9 20 ...,xlnet,5


In [ ]:
df1=df[['Generation','label1']]

In [ ]:
train_dataset, test_dataset = DataFrameDataset(
    df=df1, 
    fields=(
        ('Generation', text_field),
        ('label1', label_field)
    )
).split()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)

train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), 
    batch_sizes=(64, 64),
    sort=False,
    device=device
)

We are working with  cuda


In [ ]:
# for i in train_iter:
#   print(i.Generation.T.size())

In [ ]:
class ModelParam(object):
    def __init__(self, param_dict: dict = dict()):
        self.input_size = param_dict.get('input_size', 64)
        self.vocab_size = param_dict.get('vocab_size')
        self.embedding_dim = param_dict.get('embedding_dim', 300)
        self.target_dim = param_dict.get('target_dim', 11)
        
class MyModel(nn.Module):
    def __init__(self, model_param: ModelParam):
        super().__init__()
        self.embedding = nn.Embedding(
            model_param.vocab_size, 
            model_param.embedding_dim
        )
        self.lin = nn.Linear(
            model_param.embedding_dim*500, 
            model_param.target_dim
        )
        
    def forward(self, x):
        # print(x.size())
        features = self.embedding(x).view(x.size()[0], -1)
        # print(features.size())
        features = F.relu(features)
        features = self.lin(features)
        return features

In [ ]:
# class MyModel(nn.Module):
#     def __init__(self, model_param: ModelParam):
#         super().__init__()

#         self.embedding = nn.Embedding(
#             model_param.vocab_size, 
#             model_param.embedding_dim
#         )
#         self.lstm = nn.LSTM(input_size = model_param.embedding_dim, hidden_size = 256, num_layers = 2, dropout = 0.5, bidirectional = True)
#         self.lin = nn.Linear(
#             256 * 2, 
#             model_param.target_dim
#         )
        
#     def forward(self, x):
#         print(x.size())
#         features = self.embedding(x)
#         print(features.size())
#         features = F.relu(features)
#         output, (hidden,cell) = self.lstm(features)
#         print(hidden.size())
#         #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
#         concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
#         print(concat.size())
#         features = self.lin(concat)
#         return features

In [ ]:
# import torch.nn as nn

# class LSTM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
#                  bidirectional, dropout, pad_idx):
        
#         super().__init__()
        
#         #TO-DO
#         #1. Initialize Embedding Layer
#         self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim, padding_idx = pad_idx)
        
        
#         #2. Initialize LSTM layer
#         self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, dropout = dropout, bidirectional = bidirectional)


#         #3. Initialize a fully connected layer with Linear transformation
#         self.fc = nn.Linear(hidden_dim * n_layers, output_dim)

#         #4. Initialize Dropout
#         self.dropout = nn.Dropout(dropout)
        
#     def forward(self, text):
#         #text = [sent_len, batch_size]

#         #TO-DO
#         #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
#         embedding = self.embedding(text)
#         print(embedding.size())
#         embedding_dropout = self.dropout(embedding)
#         #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
#         output, (hidden,cell) = self.lstm(embedding_dropout)
#         #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
#         concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
#         print(concat.size())
#         hidden = self.dropout(concat)     
#         return self.fc(hidden)

In [ ]:
class MyModelWithPretrainedEmbedding(nn.Module):
    def __init__(self, model_param: ModelParam, embedding):
        super().__init__()
        self.embedding = embedding
        self.lin = nn.Linear(
            model_param.input_size * model_param.embedding_dim, 
            1
        )
        
    def forward(self, x):
        # print(x.size())
        features = self.embedding[x].reshape(x.size()[0], -1)
        # print(features.size())
        features = F.relu(features)
        features = self.lin(features)
        return features

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    # print(torch.sigmoid(preds.float()))
    rounded_preds = torch.round(torch.sigmoid(preds.float()))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    # print(correct.sum())
    # print(rounded_preds)
    return acc

In [ ]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [ ]:
pretrained_embeddings = text_field.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([83533, 300])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)


We are working with  cuda


In [ ]:
model_param = ModelParam(
    param_dict=dict(
        vocab_size=len(text_field.vocab),
        input_size=64,
    )
)

model = MyModel(model_param)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = text_field.vocab.stoi[text_field.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(300)
# model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)



# print(model.embedding.weight.data)
loss_function = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=0.01)
epochs = 10


model = model.to(device)
loss_function = loss_function.to(device)

best_valid_loss = float('inf')


for epoch in range(epochs):
    epoch_losses = list()
    for batch in train_iter:
        optimizer.zero_grad()

        prediction = model(batch.Generation.T).squeeze(1)
        # truey = torch.nn.functional.one_hot(batch.label)
        # print(prediction.size())
        # print(batch.label1.size())
        # output_rounded = torch.round(output).long()

        # print(batch.text.size())

        # print(prediction.size())

        loss = loss_function(prediction, batch.label1.long())
        train_acc = multiclass_accuracy(prediction, batch.label1)

        loss.backward()
        optimizer.step()
        train_loss = loss.item()
        epoch_losses.append(loss.item())
    
    test_losses = list()
    for batch in test_iter:
        with torch.no_grad():
            optimizer.zero_grad()
            prediction = model(batch.Generation.T)
            # truey = torch.nn.functional.one_hot(batch.label)
            loss = loss_function(prediction, batch.label1)
            valid_acc = multiclass_accuracy(prediction, batch.label1)
            valid_loss = loss.item()
            test_losses.append(loss.item())
        if loss.item() < best_valid_loss:
            torch.save(model.state_dict(), '/content/gdrive/MyDrive/task3.pt')
            best_valid_loss = loss.item()

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    # print('test loss on epoch {}: {:.3f}'.format(epoch, np.mean(test_losses)))
    # print('test accuracy on epoch {} : {:.3f}'.format(epoch, acc))

	Train Loss: 2.510 | Train Acc: 71.88%
	 Val. Loss: 2.504 |  Val. Acc: 70.97%
	Train Loss: 0.026 | Train Acc: 100.00%
	 Val. Loss: 1.990 |  Val. Acc: 67.74%
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 2.068 |  Val. Acc: 62.90%
	Train Loss: 0.001 | Train Acc: 100.00%
	 Val. Loss: 2.005 |  Val. Acc: 64.52%
	Train Loss: 0.002 | Train Acc: 100.00%
	 Val. Loss: 2.011 |  Val. Acc: 66.13%
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 2.000 |  Val. Acc: 66.13%
	Train Loss: 0.001 | Train Acc: 100.00%
	 Val. Loss: 1.991 |  Val. Acc: 66.13%
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 1.992 |  Val. Acc: 66.13%
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 1.995 |  Val. Acc: 67.74%
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 1.979 |  Val. Acc: 67.74%


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 26,709,911 trainable parameters


In [ ]:
19200/300

64.0

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_NLM(model, sentence):
    model.eval()
    # tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    # indexed = [text_field.vocab.stoi[t] for t in tokenized]
    # length = [len(indexed)]
    # tensor = torch.LongTensor(indexed).to(device)
    # tensor = tensor.unsqueeze(1)
    # length_tensor = torch.LongTensor(length)
    preds = model(sentence)
    _, preds = torch.max(preds, dim=1)
    # correct = (preds == y).float() #convert into float for division
    # prediction = torch.sigmoid(model(tensor, length_tensor))
    return preds

In [ ]:
# data = {'Generation': ['Alice', 'Bob', 'Charlie'],
#         'age': [25, 30, 35],
#         'country': ['USA', 'Canada', 'France']}

data = {'Generation':['The sun was just beginning to rise, casting a warm glow over the quiet neighborhood. Birds chirped in the trees as a light breeze rustled the leaves. In the distance, a dog barked, but otherwise, the world was still asleep. As the day slowly began to awaken, a sense of peace settled over the streets, and everything seemed possible.',
                      'His destination was Mr Trumps home at the Mar-a-Lago resort in Palm Beach, Florida, a five-hour drive from Mr Frazels home in St Augustine.','Hi I am Ganesh studying CSE at UB'],
        'label':[9,8,8]}

# create a DataFrame from the dictionary
testdf = pd.DataFrame(data)


In [ ]:
testdf

,Generation,label
0,"The sun was just beginning to rise, casting a ...",9
1,His destination was Mr Trumps home at the Mar-...,8
2,Hi I am Ganesh studying CSE at UB,8


In [ ]:
train_dataset1, test_dataset1 = DataFrameDataset(
    df=testdf, 
    fields=(
        ('Generation', text_field),
        ('label',label_field)
    )
).split()

In [ ]:
train_iter1, test_iter1 = BucketIterator.splits(
    datasets=(train_dataset1, test_dataset1), 
    batch_sizes=(64, 64),
    sort=False,
    device=device
)

In [ ]:
for i in train_iter1:
  print(i.Generation.T.size())
  print(i.label)
  print(predict_NLM(model,i.Generation.T))

torch.Size([2, 500])
tensor([8, 8], device='cuda:0')
tensor([9, 8], device='cuda:0')


In [ ]:
for i in test_iter1:
  print(i.Generation.T.size())
  print(predict_NLM(model,i.Generation.T))

torch.Size([1, 500])
tensor([8], device='cuda:0')


In [ ]:
from torchsummary import summary
